# Single-Node Bodo - Jupyter Notebook

This example shows how to do feature engineering and train models using the HPC-like platform Bodo using a notebook on a single node.
New York taxi data is used to predict how much a tip each driver will get. Both the feature engineering
and model training are parallelized across multiple cores using Bodo. This can be a straightforward way to
make Python code run faster that it would otherwise without requiring much change to the code.

The Bodo framework knows when to parallelize code based on the `%%px` at the start of cells and `@bodo.jit` function decorators. Removing those and restarting the kernel will run the code without Bodo.

**The Bodo parallel cluster in this example runs within the same Saturn Cloud resource as the notebook.** Thus, to increase the performance of the Bodo cluster you only need to increase the instance size of the Jupyter Server resource it's running on.

## Start an IPyParallel cluster

Run the following code in a cell to start an IPyParallel cluster. IPyParallel is used to interactively control a cluster of IPython processes. The variable `n` is used to specify the number of clusters based on the number of CPU cores available (up to 8 in the free Bodo Community Edition).

In [1]:
import ipyparallel as ipp
import psutil

n = min(psutil.cpu_count(logical=False), 8)

# command to create and start the local cluster
rc = ipp.Cluster(engines="mpi", n=n).start_and_connect_sync(activate=True)

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/4 [00:00<?, ?engine/s]

The following code imports bodo and verifies that the IPyParallel cluster is set up correctly:

In [1]:
%%px
import bodo

print(f"Hello World from rank {bodo.get_rank()}. Total ranks={bodo.get_size()}")

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/4 [00:00<?, ?engine/s]

%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stdout:2] Hello World from rank 2. Total ranks=4


[stdout:0] Hello World from rank 0. Total ranks=4


[stdout:1] Hello World from rank 1. Total ranks=4


[stdout:3] Hello World from rank 3. Total ranks=4


## Importing the Packages

These are the main packages we are going to work with:
 - Bodo to parallelize Python code automatically
 - Pandas to work with data
 - scikit-learn to build and evaluate regression models
 - xgboost for xgboost regressor model

In [2]:
%%px
import time

import bodo
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression, Ridge, SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

## Load and clean data

The function below will load the taxi data from a public S3 bucket into a single pandas DataFrame.


>**Note**: Bodo works best for very large datasets, so downloading the data used in the examples can take some time. Please be patient while the datasets download for each example - you will see the speed benefits of Bodo when manipulating the downloaded data.

In [3]:
%%px
@bodo.jit(distributed=["taxi"], cache=True)
def get_taxi_trips():
    start = time.time()
    taxi = pd.read_parquet(
        "s3://bodo-example-data/nyc-taxi/yellow_tripdata_2019_half.pq",
        )
    print("Reading time: ", time.time() - start)
    print(taxi.shape)
    return taxi


taxi = get_taxi_trips()
if bodo.get_rank()==0:
    display(taxi.head())

%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stdout:0] Reading time:  9.737294323645074
(42199509, 18)


[output:0]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


## Feature engineering

The data is then modified to have a set of features appropriate for machine learning. Other than
the Bodo decorators, this is the same function you would use for training without Bodo.

In [4]:
%%px
@bodo.jit(distributed=["taxi_df", "df"], cache=True)
def prep_df(taxi_df):
    """
    Generate features from a raw taxi dataframe.
    """
    start = time.time()
    df = taxi_df[taxi_df.fare_amount > 0][
        "tpep_pickup_datetime", "passenger_count", "tip_amount", "fare_amount"
    ].copy()  # avoid divide-by-zero
    df["tip_fraction"] = df.tip_amount / df.fare_amount

    df["pickup_weekday"] = df.tpep_pickup_datetime.dt.weekday
    df["pickup_weekofyear"] = df.tpep_pickup_datetime.dt.weekofyear
    df["pickup_hour"] = df.tpep_pickup_datetime.dt.hour
    df["pickup_week_hour"] = (df.pickup_weekday * 24) + df.pickup_hour
    df["pickup_minute"] = df.tpep_pickup_datetime.dt.minute
    df = (
        df[
            "pickup_weekday",
            "pickup_weekofyear",
            "pickup_hour",
            "pickup_week_hour",
            "pickup_minute",
            "passenger_count",
            "tip_fraction",
        ]
        .astype(float)
        .fillna(-1)
    )
    print("Data preparation time: ", time.time() - start)
    print(df.head())
    return df


taxi_feat = prep_df(taxi)

%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stdout:0] Data preparation time:  5.146262661548462
   pickup_weekday  pickup_weekofyear  pickup_hour  pickup_week_hour  \
0             1.0                1.0          0.0              24.0   
1             1.0                1.0          0.0              24.0   
2             4.0               51.0         13.0             109.0   
3             2.0               48.0         15.0              63.0   
4             2.0               48.0         15.0              63.0   

   pickup_minute  passenger_count  tip_fraction  
0           46.0              1.0      0.235714  
1           59.0              1.0      0.071429  
2           48.0              3.0      0.000000  
3           52.0              5.0      0.000000  
4           56.0              5.0      0.000000  


[stdout:1] Empty DataFrame
Columns: [pickup_weekday, pickup_weekofyear, pickup_hour, pickup_week_hour, pickup_minute, passenger_count, tip_fraction]
Index: []


[stdout:3] Empty DataFrame
Columns: [pickup_weekday, pickup_weekofyear, pickup_hour, pickup_week_hour, pickup_minute, passenger_count, tip_fraction]
Index: []


[stdout:2] Empty DataFrame
Columns: [pickup_weekday, pickup_weekofyear, pickup_hour, pickup_week_hour, pickup_minute, passenger_count, tip_fraction]
Index: []


The data is then split into X and y sets as well as training and testing using the scikit-learn functions. Again bodo is used to increase the speed at which it runs.

In [5]:
%%px
@bodo.jit(distributed=["taxi_feat", "X_train", "X_test", "y_train", "y_test"])
def data_split(taxi_feat):
    X_train, X_test, y_train, y_test = train_test_split(
        taxi_feat[
            "pickup_weekday",
            "pickup_weekofyear",
            "pickup_hour",
            "pickup_week_hour",
            "pickup_minute",
            "passenger_count",
        ],
        taxi_feat["tip_fraction"],
        test_size=0.3,
        train_size=0.7,
        random_state=42,
    )
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = data_split(taxi_feat)

%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

## Training the model


Below we'll train four distinct linear models on the data, all using Bodo for faster performance.
We'll train the models to predict the `tip_fraction` variable and evaluate these models against the test set using RMSE.

#### Linear regression

In [6]:
%%px
@bodo.jit(distributed=["X_train", "y_train", "X_test", "y_test"], cache=True)
def lr_model(X_train, y_train, X_test, y_test):
    start = time.time()
    lr = LinearRegression()
    lr_fitted = lr.fit(X_train, y_train)
    print("Linear Regression fitting time: ", time.time() - start)

    start = time.time()
    lr_preds = lr_fitted.predict(X_test)
    print("Linear Regression prediction time: ", time.time() - start)
    print(mean_squared_error(y_test, lr_preds, squared=False))


lr_model(X_train, y_train, X_test, y_test)

[stderr:0] /tmp/ipykernel_18146/3303269868.py:14: BodoWarning: Data is distributed so Bodo will fit model with SGD solver optimization (SGDRegressor)
  lr_model(X_train, y_train, X_test, y_test)
/opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stdout:0] Linear Regression fitting time:  23.256143451334992
Linear Regression prediction time:  0.23284950440529428
18.481945248580985


[stderr:1] /opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


[stderr:2] /opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


[stderr:3] /opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


#### Ridge regression

In [7]:
%%px
@bodo.jit(distributed=["X_train", "y_train", "X_test", "y_test"])
def rr_model(X_train, y_train, X_test, y_test):
    start = time.time()
    rr = Ridge()
    rr_fitted = rr.fit(X_train, y_train)
    print("Ridge fitting time: ", time.time() - start)

    start = time.time()
    rr_preds = rr_fitted.predict(X_test)
    print("Ridge prediction time: ", time.time() - start)
    print(mean_squared_error(y_test, rr_preds, squared=False))


rr_model(X_train, y_train, X_test, y_test)

%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stderr:0] /tmp/ipykernel_18146/2368029932.py:14: BodoWarning: Data is distributed so Bodo will fit model with SGD solver optimization (SGDRegressor)
  rr_model(X_train, y_train, X_test, y_test)
/opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(


[stdout:0] Ridge fitting time:  20.791850138289647
Ridge prediction time:  0.049831843648007634
18.48113144281467


[stderr:1] /opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(


[stderr:2] /opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(


[stderr:3] /opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(


#### Lasso regression

In [8]:
%%px
@bodo.jit(distributed=["X_train", "y_train", "X_test", "y_test"])
def lsr_model(X_train, y_train, X_test, y_test):
    start = time.time()
    lsr = Lasso()
    lsr_fitted = lsr.fit(X_train, y_train)
    print("Lasso fitting time: ", time.time() - start)

    start = time.time()
    lsr_preds = lsr_fitted.predict(X_test)
    print("Lasso prediction time: ", time.time() - start)
    print(mean_squared_error(y_test, lsr_preds, squared=False))


lsr_model(X_train, y_train, X_test, y_test)

[stderr:0] /tmp/ipykernel_18146/2164705375.py:14: BodoWarning: Data is distributed so Bodo will fit model with SGD solver optimization (SGDRegressor)
  lsr_model(X_train, y_train, X_test, y_test)
/opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stdout:0] Lasso fitting time:  24.764697136187806
Lasso prediction time:  0.049428549449203274
18.481029374118005


[stderr:1] /opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


[stderr:3] /opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


[stderr:2] /opt/conda/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


#### SGD regressor model

In [9]:
%%px
@bodo.jit(distributed=["X_train", "y_train", "X_test", "y_test"])
def sgdr_model(X_train, y_train, X_test, y_test):
    start = time.time()
    sgdr = SGDRegressor(max_iter=100, penalty="l2")
    sgdr_fitted = sgdr.fit(X_train, y_train)
    print("SGDRegressor fitting time: ", time.time() - start)

    start = time.time()
    sgdr_preds = sgdr_fitted.predict(X_test)
    print("SGDRegressor prediction time: ", time.time() - start)
    print(mean_squared_error(y_test, sgdr_preds, squared=False))


sgdr_model(X_train, y_train, X_test, y_test)

%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stdout:0] SGDRegressor fitting time:  23.010143700188564
SGDRegressor prediction time:  0.049575961667869706
18.48163039072187


## Stopping the cluster

When you're done using the parallel cluster you can shut it down with a single command. Note that since the cluster is running within the same Jupyter Server resource as the notebook, there is no change to what hardware is running after you stop it (since the resource is still on).

In [10]:
# command to stop the cluster
rc.cluster.stop_cluster_sync()